# Traffic Forecasting using Graph Neural Networks

In [1]:
#install pytorch geometric
!pip install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cpu.html
!pip install torch-geometric

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from google.colab import drive
import scipy.sparse as sp
import torch
import torch.utils.data
from torch_geometric.utils.convert import from_networkx
import networkx as nx
import math

from sklearn import preprocessing
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 KB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 35.0 MB/s eta 0:00:00
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773302 sha256=093c0c4762550aeaf5c3812b0b2bd4579b7d74a240777c7a78c861745a4ca196
  Stored in directory: /root/.cache/pip/wheels/59/a3/20/198928106d3169865ae73afcbd3d3d1796cf6b429b55c65378
Successfully built torch-geometric
  Attempting uninstall: psutil
    Found existing installation:

Mounted at /content/drive


Get dataset files (upload correspondent *adj.npz* file into colab content folder)

In [4]:
#METR-LA dataset without saturdays and sundays
#!cp /content/drive/MyDrive/modified-METRLA.csv /content/dataset.csv

#Pems-bay dataset
!cp /content/drive/MyDrive/pems-bay.csv /content/dataset.csv

#Pemsd7-m dataset
#!cp /content/drive/MyDrive/pemsd7-m.csv /content/dataset.csv

In [5]:
#dataset path
dataset_path = '/content/dataset.csv'
npz_path = '/content/adj.npz'

Clear dataset removing first column of date info if necessary and convert values into numpy array

In [22]:
# read the CSV file
df = pd.read_csv(dataset_path)

# drop the first column if modified METR-LA dataset
#df = df.drop(df.columns[0], axis=1)
dataset = df.values
dataset.astype(np.float128)

# save the modified dataframe
#df.to_csv(dataset_path, index=False)

array([[71.6, 67.5, 70.6, ..., 68.4, 70.8, 67.4],
       [71.6, 67.6, 70.2, ..., 68.4, 70.5, 67.9],
       [71.1, 67.5, 70.3, ..., 68.4, 70.8, 67.6],
       ...,
       [71.4, 66.9, 68.1, ..., 68.4, 71.6, 66.6],
       [72.2, 66.5, 68. , ..., 68.7, 71.6, 68.4],
       [71.5, 66.2, 68.4, ..., 68.7, 71.6, 68. ]], dtype=float128)

Get data from *adj.npz* file

In [23]:
# convert adj to Compressed Sparse Column format
adj = sp.load_npz(npz_path)
n = adj.shape[0]
adj = adj.tocsc()
adj = adj.todense()

### Create correspondent graph using networkx package

In [24]:
#create graph
G = nx.Graph()

for i in range(adj.shape[0]):
  for j in range(adj.shape[0]):
    if adj[i, j] != 1 and adj[i, j] != 0 and not G.has_edge(i, j):
      G.add_edge(i, j, length=adj[i, j])

In [26]:
#plot it
#%matplotlib inline
#import matplotlib.pyplot as plt
#nx.draw(G)

In [25]:
#set sensor number and measurements values attributes to all nodes
nx.set_node_attributes(G, 0, 'x')
nx.set_node_attributes(G, 0, 'y')

rate = 0.8
x_size = int(dataset.shape[0] * rate)

for i in range(dataset.shape[1]):
  G.nodes[i]['x'] = dataset[:x_size, i]
  G.nodes[i]['y'] = dataset[x_size:, i]

In [26]:
#convert into torch geometric data
data = from_networkx(G)
print(type(data))

print(data)
print(data.x)
print(data.edge_index)

<class 'torch_geometric.data.data.Data'>
Data(x=[325, 41692], edge_index=[2, 38562], y=[325, 10423], length=[38562])
tensor([[71.6000, 71.6000, 71.1000,  ..., 71.2000, 72.7000, 71.9000],
        [70.6000, 70.2000, 70.3000,  ..., 54.7000, 55.0000, 55.4000],
        [67.5000, 67.4000, 68.0000,  ..., 53.6000, 56.2000, 56.8000],
        ...,
        [70.2000, 70.6000, 70.3000,  ..., 58.8000, 60.7000, 58.1000],
        [64.9000, 64.9000, 64.7000,  ..., 64.1000, 65.0000, 66.3000],
        [65.0000, 65.0000, 65.0000,  ..., 29.3000, 29.6000, 30.3000]],
       dtype=torch.float64)
tensor([[  0,   0,   0,  ..., 324, 324, 324],
        [  1,   2,   3,  ..., 321, 322, 323]])


In [27]:
# Split the data 
train_ratio = 0.2
num_nodes = data.x.shape[0]
num_train = int(num_nodes * train_ratio)
idx = [i for i in range(num_nodes)]

np.random.shuffle(idx)
train_mask = torch.full_like(data.y, False, dtype=bool)
train_mask[idx[:num_train]] = True
test_mask = torch.full_like(data.y, False, dtype=bool)
test_mask[idx[num_train:]] = True

### GCN Model

In [28]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(data.num_features, hidden_channels*2)
        self.conv2 = GCNConv(hidden_channels*2, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, data.y.shape[1])

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        return x

In [29]:
model = GCN(hidden_channels=100)
print(model)
model = model.double()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.MSELoss()

GCN(
  (conv1): GCNConv(41692, 200)
  (conv2): GCNConv(200, 100)
  (conv3): GCNConv(100, 10423)
)


Define train and test functions

In [30]:
def train():
      model.train()
      optimizer.zero_grad()
      out = model(data.x.double(), data.edge_index)
      loss = criterion(out[train_mask], data.y[train_mask])
      loss.backward()
      optimizer.step()
      return loss

def test():
      model.eval()
      out = model(data.x.double(), data.edge_index)
      test_error = (out[test_mask] - data.y[test_mask])**2
      test_err = int(test_error.sum()) / int(test_mask.sum())
      return test_err

Train and test the model

In [31]:
for epoch in range(1, 50):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

test_err = test()
print(f'Test Loss: {test_err:.4f}')

Epoch: 001, Loss: 3998.0954
Epoch: 002, Loss: 5355616.2537
Epoch: 003, Loss: 1215847.3969
Epoch: 004, Loss: 16395356.1879
Epoch: 005, Loss: 1245662.5652
Epoch: 006, Loss: 1027698.8634
Epoch: 007, Loss: 88663.3594
Epoch: 008, Loss: 175363.0576
Epoch: 009, Loss: 20013.6418
Epoch: 010, Loss: 133599.8775
Epoch: 011, Loss: 7320.2892
Epoch: 012, Loss: 38568.5132
Epoch: 013, Loss: 30885.4892
Epoch: 014, Loss: 5221.9606
Epoch: 015, Loss: 4226.2148
Epoch: 016, Loss: 4300.2465
Epoch: 017, Loss: 4028.0598
Epoch: 018, Loss: 3955.3699
Epoch: 019, Loss: 3631.9630
Epoch: 020, Loss: 65054.6832
Epoch: 021, Loss: 2472.6717
Epoch: 022, Loss: 3952.7877
Epoch: 023, Loss: 3952.7767
Epoch: 024, Loss: 3952.7582
Epoch: 025, Loss: 3952.7330
Epoch: 026, Loss: 3952.7013
Epoch: 027, Loss: 3952.6637
Epoch: 028, Loss: 3952.6207
Epoch: 029, Loss: 3952.5725
Epoch: 030, Loss: 3952.5196
Epoch: 031, Loss: 3952.4623
Epoch: 032, Loss: 3952.4008
Epoch: 033, Loss: 3952.3354
Epoch: 034, Loss: 3952.2663
Epoch: 035, Loss: 3952.